In [18]:
import os
import urllib
import json
import pandas as pd
from pandas import json_normalize
from datetime import datetime
from datetime import date
import time

In [36]:
# GitHub API unauthenticated requests rate limit = 10 requests per minute.
orgs = ["nhsx","111Online","NHSDigital","nhsconnect","nhsengland"]

In [26]:
# last modified datex
data_updated = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print(data_updated)

31/03/2021 12:47:51


In [34]:
df = pd.DataFrame()
for org in orgs:
    data = [1]
    page = 1
    while bool(data) is True:
        url = "https://api.github.com/orgs/"+str(org)+"/repos?page="+str(page)+"&per_page=100"
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        flat_data = json_normalize(data)
        df = df.append(flat_data)
        page = page + 1
        time.sleep(10) # Sleep to avoid rate limit
df.columns = df.columns.str.replace('.', '_')
df.head()

,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,owner_site_admin,license_key,license_name,license_spdx_id,license_url,license_node_id,permissions_admin,permissions_push,permissions_pull,license
0,57110085,MDEwOlJlcG9zaXRvcnk1NzExMDA4NQ==,gpconnect-demonstrator,nhsconnect/gpconnect-demonstrator,False,https://github.com/nhsconnect/gpconnect-demons...,GP Connect demonstrator is a reference impleme...,False,https://api.github.com/repos/nhsconnect/gpconn...,https://api.github.com/repos/nhsconnect/gpconn...,...,False,apache-2.0,Apache License 2.0,Apache-2.0,https://api.github.com/licenses/apache-2.0,MDc6TGljZW5zZTI=,False,False,True,NaN
1,63938241,MDEwOlJlcG9zaXRvcnk2MzkzODI0MQ==,gpconnect,nhsconnect/gpconnect,False,https://github.com/nhsconnect/gpconnect,GP Connect API specification,False,https://api.github.com/repos/nhsconnect/gpconnect,https://api.github.com/repos/nhsconnect/gpconn...,...,False,apache-2.0,Apache License 2.0,Apache-2.0,https://api.github.com/licenses/apache-2.0,MDc6TGljZW5zZTI=,False,False,True,NaN
2,64459031,MDEwOlJlcG9zaXRvcnk2NDQ1OTAzMQ==,documentation-theme-jekyll,nhsconnect/documentation-theme-jekyll,False,https://github.com/nhsconnect/documentation-th...,A Jekyll-based theme designed for documentatio...,True,https://api.github.com/repos/nhsconnect/docume...,https://api.github.com/repos/nhsconnect/docume...,...,False,NaN,NaN,NaN,NaN,NaN,False,False,True,NaN
3,64459559,MDEwOlJlcG9zaXRvcnk2NDQ1OTU1OQ==,gpconnect-fhir,nhsconnect/gpconnect-fhir,False,https://github.com/nhsconnect/gpconnect-fhir,GP Connect FHIR assets,False,https://api.github.com/repos/nhsconnect/gpconn...,https://api.github.com/repos/nhsconnect/gpconn...,...,False,apache-2.0,Apache License 2.0,Apache-2.0,https://api.github.com/licenses/apache-2.0,MDc6TGljZW5zZTI=,False,False,True,NaN
4,65154570,MDEwOlJlcG9zaXRvcnk2NTE1NDU3MA==,cda-to-fhir-eol,nhsconnect/cda-to-fhir-eol,False,https://github.com/nhsconnect/cda-to-fhir-eol,End of Life - FHIR Document Solution,False,https://api.github.com/repos/nhsconnect/cda-to...,https://api.github.com/repos/nhsconnect/cda-to...,...,False,apache-2.0,Apache License 2.0,Apache-2.0,https://api.github.com/licenses/apache-2.0,MDc6TGljZW5zZTI=,False,False,True,NaN


In [35]:
# Group the data frame by month and item and extract a number of stats from each group
df_group = df.groupby(
    ['owner_login']
).agg(
    {
        # find the number of open repos
        'name': "count",
        'size': "sum",
        'stargazers_count': "sum",
        'forks_count': "sum",
        'open_issues_count': "sum",
        'license_name': lambda x:x.value_counts().index[0],
        'language': lambda x:x.value_counts().index[0]
    }
)
df_group

,name,size,stargazers_count,forks_count,open_issues_count,license_name,language
owner_login,,,,,,,
nhsconnect,178,3193113,243,198,406,Apache License 2.0,CSS
nhsengland,23,36298,14,38,80,MIT License,Python


In [33]:
df_html = df_group.head(10).to_html(classes='summary')

In [ ]:
# Write HTML String to file.html
with open("_includes/table1.html", "w") as file:
    file.write(df_html)

In [ ]:
import plotly 
# Use init_notebook_mode() to view the plots in jupyter notebook
plotly.offline.init_notebook_mode()
from plotly.graph_objs import Scatter,Layout,Bar

trace1 = Bar(x=data_canada['year'],y=data_canada['pop'])
    
# Create chart 
plotly.offline.iplot(
    {
        "data": [trace1],
        "layout": Layout(
            title="<b>Simple Bar Chart - Year Vs Population</b>",
            xaxis=dict(
                title="<b>Year</b>",
                zeroline=False,
                gridcolor="rgb(183,183,183)",
                showline=True,
            ),
            yaxis=dict(
                title="<b>Population</b>",
                gridcolor="rgb(183,183,183)",
                zeroline=False,
                showline=True,
            ),
            font=dict(family="Courier New, monospace", size=12, color="rgb(0,0,0)"),
        ),
    }
)


In [ ]:
import plotly.io as pio


pio.write_html(fig, file='index.html', auto_open=True)